In [1]:
import pandas as pd
import os
import numpy as np
import altair as alt
from vega_datasets import data
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

## Cleaning and exploration

#### Join datasets

In [2]:
def join_datasets():
    folder_path = '/Users/paulacadena/CAPP30239-SP/data'
    dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path, encoding='ISO-8859-1')
                dataframes.append(df)
            except UnicodeDecodeError:
                print(f'Could not decode {filename}. Skipping.')

    return pd.concat(dataframes, ignore_index=True)

#### Clean and unify

In [3]:
def clean_dataset():
    world_bank = join_datasets()

    #Correctly name missing values
    world_bank.replace('..', np.nan, inplace=True)
    #Drop missing values in identificating columns
    world_bank.dropna(subset=['Series Code', 'Country Code', 'Series Name'], inplace=True)
    #Drop wrongly identified country codes
    world_bank = world_bank[world_bank['Country Code'].str.len() <= 3]

    return world_bank

##### For easier use in visualizations

In [4]:
def wide_long_wb():
    world_bank = clean_dataset()

    # Identify the columns to transform
    value_vars = [col for col in world_bank.columns if 'YR' in col]

    # Melt the DataFrame
    long_format = pd.melt(world_bank, 
                        id_vars=[col for col in world_bank.columns if col not in value_vars],
                        value_vars=value_vars, 
                        var_name='YEAR', 
                        value_name='Value')

    # Extract the year from the 'YEAR' column
    long_format['YEAR'] = long_format['YEAR'].str.extract(r'(\d{4})')[0].astype(int)
    #Change Value column to numeric
    long_format['Value'] = pd.to_numeric(long_format['Value'], errors='coerce')

    return long_format

##### To add more details

In [5]:
def add_continents(df):
    continents = pd.read_csv('/Users/paulacadena/CAPP30239-SP/data/old/continents2.csv', 
                             usecols = ['alpha-3','region','sub-region','country-code'])
    df = df.merge(continents, left_on = 'Country Code', right_on= 'alpha-3', how='left')
    df['country-code'] = df['country-code'].fillna(-1).astype(int)
    df.drop(columns='alpha-3',inplace = True)
    df.loc[df['Country Name'] == 'Kosovo','region'] = 'Europe'
    df.loc[df['Country Name'] == 'Channel Islands','region'] = 'Europe'
    df['region'] = df['region'].fillna('Aggregated data')
    return df

In [6]:
def add_decade(df):
    df['YEAR'] = df['YEAR'].astype(int)
    df['DECADE'] = (df['YEAR'] // 10) * 10
    return df

In [7]:
def world_bank_complete():
    df = wide_long_wb()
    df = add_continents(df)
    df = add_decade(df)
    df = df.rename(columns = {'Country Name':'Country', 'Series Name': 'Series', 
                         'YEAR': 'Year', 'region': 'Region', 'sub-region': 'Sub-region'})
    return df

In [8]:
world_bank = world_bank_complete()

#### Exploration

In [9]:
world_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8018304 entries, 0 to 8018303
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Country       object 
 1   Country Code  object 
 2   Series        object 
 3   Series Code   object 
 4   Year          int64  
 5   Value         float64
 6   country-code  int64  
 7   Region        object 
 8   Sub-region    object 
 9   DECADE        int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 611.7+ MB


In [16]:
world_bank['Series'].unique()

array(['GDP (constant 2015 US$)',
       'Average working hours of children, study and work, ages 7-14 (hours per week)',
       'Average working hours of children, study and work, female, ages 7-14 (hours per week)',
       'Average working hours of children, study and work, male, ages 7-14 (hours per week)',
       'Average working hours of children, working only, ages 7-14 (hours per week)',
       'Average working hours of children, working only, female, ages 7-14 (hours per week)',
       'Average working hours of children, working only, male, ages 7-14 (hours per week)',
       'Child employment in agriculture (% of economically active children ages 7-14)',
       'Child employment in agriculture, female (% of female economically active children ages 7-14)',
       'Child employment in agriculture, male (% of male economically active children ages 7-14)',
       'Child employment in manufacturing (% of economically active children ages 7-14)',
       'Child employment in manufact

In [19]:
def histogram_missing(variable, bins):
    
    # Group by the selected variable and count missing values in 'Value' column
    missing_values = world_bank.groupby(variable)['Value'].apply(lambda x: x.isna().sum()).reset_index()
    missing_values.columns = [variable, 'MissingValues']
    
    histogram = alt.Chart(missing_values).mark_bar().encode(
        alt.X('MissingValues:Q', bin=alt.Bin(maxbins=bins), title='Number of Missing Values'),
        alt.Y('count()', title=f'Count of {variable}')
    ).properties(
        title=f'Histogram of Missing Values by {variable}',
        width=800,
        height=400
    )

    histogram.display()
    return missing_values

In [20]:
histogram_missing('Series',40)

alt.Chart(...)

,Series,MissingValues
0,Access to clean fuels and technologies for coo...,11573
1,Access to clean fuels and technologies for coo...,11573
2,Access to clean fuels and technologies for coo...,11573
3,Access to electricity (% of population),9411
4,"Access to electricity, rural (% of rural popul...",9825
...,...,...
466,"Wage and salaried workers, male (% of male emp...",9505
467,"Wage and salaried workers, total (% of total e...",9505
468,Wanted fertility rate (births per woman),16701
469,Women who were first married by age 15 (% of w...,16611


In [12]:
histogram_missing('Country',40)

alt.Chart(...)

## Visualizations

1. Map

In [54]:
def world_map(series):
    world_bank = world_bank_complete()
    
    # Filter data based on the selected series
    world_bank = world_bank[world_bank['Series'] == series]

    # Filtering the first year and last year available data for each country
    first_data = world_bank.sort_values(by=['Country', 'Year']).groupby('Country').first().reset_index()
    last_data = world_bank.sort_values(by=['Country', 'Year'], ascending=False).groupby('Country').first().reset_index()

    # Calculate the min and max for the legend
    combined_data = pd.concat([first_data, last_data])
    min_value = combined_data['Value'].min()
    max_value = combined_data['Value'].max()

    # Load important world map
    source = alt.topo_feature(data.world_110m.url, 'countries')
    background = alt.Chart(source).mark_geoshape(fill='lightgray', stroke='white')

    # First map layer for first year available data
    first_map = (
        background
        + alt.Chart(source)
        .mark_geoshape(stroke='black', strokeWidth=0.15)
        .encode(
            color=alt.Color(
                'Value:Q',
                scale=alt.Scale(
                    domain=[min_value, 0, max_value],
                    range=['red', 'white', 'green']
                ),
                legend=alt.Legend()
            ),
            tooltip=[
                alt.Tooltip('Country:N', title='Country'),
                alt.Tooltip('Year:O', title='Year'),
                alt.Tooltip('Value:Q', title='Value')
            ]
        )
        .transform_lookup(
            lookup='id',
            from_=alt.LookupData(first_data, 'country-code', ['Value', 'Country', 'Year'])
        )
    ).properties(title=f'{series}: First Available Year', width=700, height=400).project('equalEarth')

    # Second map layer for last year available data
    last_map = (
        background
        + alt.Chart(source)
        .mark_geoshape(stroke='black', strokeWidth=0.15, fillOpacity=1)
        .encode(
            color=alt.Color(
                'Value:Q',
                scale=alt.Scale(
                    domain=[min_value, 0, max_value],
                    range=['red', 'white', 'green']
                ),
                legend=alt.Legend()
            ),
            tooltip=[
                alt.Tooltip('Country:N', title='Country'),
                alt.Tooltip('Year:O', title='Year'),
                alt.Tooltip('Value:Q', title='Value')
            ]
        )
        .transform_lookup(
            lookup='id',
            from_=alt.LookupData(last_data, 'country-code', ['Value', 'Country', 'Year'])
        )
    ).properties(title=f'{series}: Last Available Year', width=700, height=400).project('equalEarth')

    final_map = alt.vconcat(
        first_map,
        last_map
    ).configure_view(strokeWidth=0)

    return final_map

world_map('Population growth (annual %)')

alt.VConcatChart(...)

2. Double axis

3. Heatmap

In [ ]:
def heat_map_series():
    world_bank = world_bank_complete()
    # Create a heatmap
    heatmap = alt.Chart(world_bank).mark_rect().encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=-45, labelFontSize=10)),
        y=alt.Y('Country:N', title='Country', sort='-x'),
        color=alt.Color('Value:Q', scale=alt.Scale(scheme='redyellowgreen'), title='Value'),
        tooltip=['Country', 'Year', 'Value', 'Series']
    ).properties(
        width=250,  # Smaller width for individual heatmaps in the facet
        height=600  # Adjust height for readability
    ).facet(
        row=alt.Row('Series:N', title='Indicator', header=alt.Header(labelFontSize=12, titleFontSize=14)),
        spacing=5  # Add space between facets
    ).resolve_scale(
        color='independent'  # Allows each heatmap to have its own color scale
    )

    heatmap.display()
heat_map_series()

4. Bars / Stacked

In [58]:
def top_bottom_bars(series, year):
    # Load complete world_bank dataset
    world_bank = world_bank_complete()
    # Filter data based on the provided series
    world_bank = world_bank[world_bank['Series'] == series]
    world_bank = world_bank[world_bank['Year'] == year]
    
    # Drop any missing values in 'Value' to avoid issues in sorting
    world_bank = world_bank.dropna(subset=['Value'])
    
    # Sort by 'Value' and select top 10 and bottom 10
    top_10 = world_bank.nlargest(10, 'Value')
    bottom_10 = world_bank.nsmallest(10, 'Value')
    
    # Concatenate top and bottom 10 data
    top_bottom = pd.concat([top_10, bottom_10])
    
    # Create a bar chart
    bar_chart = alt.Chart(top_bottom).mark_bar().encode(
        x=alt.X('Value:Q', title=f'{series} Value'),
        y=alt.Y('Country:N', sort='-x', title='Country'),
        color=alt.condition(
            alt.datum['Value'] > 0,
            alt.value('green'),
            alt.value('red')
        ),
        tooltip=['Country', 'Value']
    ).properties(
        title=f'{series}: Top 10 and Bottom 10 Countries',
        width=700,
        height=400
    )

    return bar_chart

top_bottom_bars('Population growth (annual %)', 2023)

alt.Chart(...)

5. Lines

6. Area

7. Scatter plot (faceted)

8. Trends

9. Population Pyramid